In [25]:
import numpy as np
import math
import torch
import torch.nn as nn
from torch.optim import Adam
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

x_dim = 2
hiddens = [x_dim + 1, 32, 64, 1]
test_size = 0.2
valid_size = 0.125
batch_size = 10
seq_len = 10
l = 0.1
c_hiddens = [x_dim + 1, 32, 64, 1]
g_hidden_size = 64
g_num_layers = 2
d_hidden_size = 64
d_num_layers = 2
gan_epochs = 3000

def to_tensor(z, x, y=None):
    if torch.is_tensor(x):
        zx = torch.cat([z, x], dim=1)
    else:
        zx = np.concatenate([z, x], axis=1)
        zx = torch.FloatTensor(zx)
    if isinstance(y, np.ndarray):
        y = torch.FloatTensor(y)
        return zx, y
    return zx

class TrueModel(nn.Module):

    def __init__(self, hiddens, seed=0):
        super().__init__()
        layers = []
        for in_dim, out_dim in zip(hiddens[:-1], hiddens[1:]):
            layers.append(nn.Linear(in_dim, out_dim))
            layers.append(nn.ReLU(inplace=True))
        layers.pop()
        layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)

        self.loss_fn = nn.BCELoss()
        self.optim = Adam(self.parameters())

    def forward(self, zx):
        return self.model(zx)

    def predict(self, z, x):
        zx = to_tensor(z, x)
        pred = self(zx)
        pred_y = pred.detach().round().cpu().numpy()
        return pred_y

    def fit(self, z, x, y, patience=10):
        zx, y = to_tensor(z, x, y)

        epoch, counter = 0, 0
        best_loss = float('inf')
        while True:
            pred = self(zx)
            loss = self.loss_fn(pred, y)

            self.optim.zero_grad()
            loss.backward()
            self.optim.step()
            
            epoch += 1
            if loss.item() <= best_loss:
                torch.save(self.state_dict(), self.path)
                best_loss = loss.item()
                counter = 0
            else:
                counter += 1
                if counter == patience:
                    break
        print(f"TrueModel Fit Done in {epoch} epochs!")

    def sample(self, s, x, scale=0.8):
        sx = to_tensor(s, x)
        prob = self(sx)
        y = torch.bernoulli(prob * scale)
        return y.detach().cpu().numpy()

In [64]:

def gen_initial_data(n, seed = 0):
    np.random.seed(seed)
    torch.manual_seed(seed)
    s0 = torch.bernoulli(torch.empty(n,1).uniform_(0,1)).numpy()
    x0 = np.random.randn(n, 1) + np.sin(s0)
    z0 = np.cos(x0) + np.random.randn(n, 1)
    y = torch.bernoulli(torch.from_numpy(1 /(1+  np.exp(-x0 +z0))))
    return torch.from_numpy(s0), to_tensor(x0, z0), y

def sequential_data(s0, x0, y0, seq_len, hiddens, l, seed=0):
    n = s0.size()[0]
    model = TrueModel(hiddens, seed)
    sx = to_tensor(s0, x0)
    sx.requires_grad = True
    sx = sx.to(dtype=torch.float32)
    prob = model(sx)
    loss = nn.BCELoss()(prob, torch.ones_like(prob))
    loss.backward()
    x = x0
    y= y0
    prevx = x.numpy()
    prevy = y
    s0 = s0.numpy()
    nx = np.empty_like(s0)
    nz = np.empty_like(s0)
    ny = np.empty_like(s0)
    for i in range(1, seq_len):
        loss = nn.BCELoss()(prob, torch.ones_like(prob))
        delta_y = prevy*loss
        for j in range(n):
            nx[j] = np.random.randn() + np.sin(s0[j]) + l*(prevx[j][0] - int(delta_y[j]))
            nz[j] = np.cos(nx[j]) + np.random.randn() + l*(prevx[j][1] - int(delta_y[j]))
        ny = torch.bernoulli(torch.from_numpy(1 /(1+  np.exp(-nx +nz))))
        prevx = to_tensor(nx, nz)
        x = torch.cat((x, prevx),0)
        y = torch.cat((y, ny),0)
        prevx = prevx.numpy()
        prevy = ny
    x = np.array(x, dtype=np.float32).reshape((n, seq_len, 2))
    y = np.array(y, dtype=np.int32).reshape(n, seq_len, 1)
    return x, y
        

In [92]:
s0, x0, y0 = gen_initial_data(5,0)
x, y = sequential_data(s0, x0, y0, seq_len, hiddens, l, seed=0)

In [98]:
print(x.shape)
print(y.shape)
print(s0.size()[0])

(5, 10, 2)
(5, 10, 1)
5


In [95]:
print(x0)
print(x[0])

tensor([[ 1.7641, -1.1693],
        [ 1.2416,  1.2733],
        [ 0.9787,  0.4067],
        [ 2.2409, -0.7243],
        [ 1.8676,  0.1182]])
[[ 1.7640524  -1.1693332 ]
 [ 1.2416282   1.2733443 ]
 [ 0.978738    0.406713  ]
 [ 2.2408931  -0.7242808 ]
 [ 1.867558    0.11817354]
 [ 0.32044882  2.2864342 ]
 [ 1.7266716   0.09376465]
 [ 0.541737    1.23116   ]
 [ 1.7181684  -0.4244255 ]
 [ 0.4998235   0.03538879]]


In [55]:
print(x[0])

[[ 1.7640524   1.2416282 ]
 [ 0.978738    2.2408931 ]
 [ 1.867558   -0.13580686]
 [ 1.7915595   0.6901138 ]
 [ 0.73825216  0.41059852]
 [ 0.9855146   2.2957444 ]
 [ 0.7610377   0.12167501]
 [ 0.44386324  0.33367434]
 [ 2.33555    -0.20515826]
 [ 0.3130677  -0.85409576]]


In [36]:

s_train, s_test, x_train, x_test, y_train, y_test = train_test_split(s0, np.transpose(np.array(x, dtype=np.float32)).reshape((10000, 2, seq_len)), y, test_size=test_size, random_state=10)


[[ 1.7640524   1.2416282   0.978738    2.2408931   1.867558   -0.13580686
   1.7915595   0.6901138   0.73825216  0.41059852]
 [ 0.9855146   2.2957444   0.7610377   0.12167501  0.44386324  0.33367434
   2.33555    -0.20515826  0.3130677  -0.85409576]]


In [32]:
def visualize_data(data, label, step, sample_size=10000):
    batch, n_dim, steps = data.shape

(8000, 2, 10)
